### Take time series, associated RV measurement uncertainties, and planet orbital parameters and calculate the generalized (aka expected) Fisher Information Matrix (FIM)

Let's start with the most trivial case of circular orbit and white noise

In [1]:
import numpy as np 
import scipy
print(np.__version__)
print(scipy.__version__)

import pandas as pd
import exoplanet
import astropy 
import pymc3
import pymc3_ext
import celerite2

1.21.0
1.7.3


In [27]:
param_list = ['K','P','T0']

def partial_mu_partial_K(t, P, T0):
    """
    Calculate the partial derivative of the model w.r.t. semi-amplitude, K
    
    Inputs: 
    - t: time series of length N observations; np.array [day]
    - P: planet period [days]
    - T0: mean transit time [day]
    
    Output: 
    - array of partial derivatives of length N, where N is number of observations (ie. length of the model vector)
    """
    
    normalized_t = t-T0
    #print(normalized_t)
    arg = (2*np.pi/P) * normalized_t
    #print(arg)
    return -np.sin(arg)

def partial_mu_partial_P(t, K, P, T0):
    """
    Calculate the partial derivative of the model w.r.t. period, P
    
    Inputs: 
    - t: time series of length N observations; np.array [day]
    - K: RV semi-amplitude [cm/s]
    - P: planet period [days]
    - T0: mean transit time [day]
    
    Output: 
    - array of partial derivatives of length N, where N is number of observations (ie. length of the model vector)
    """
    
    normalized_t = t-T0
    #print(normalized_t)
    arg = (2*np.pi/P) * normalized_t
    #print(arg)
    return -K * np.cos(arg) * arg/P

def partial_mu_partial_T0(t, K, P, T0):
    """
    Calculate the partial derivative of the model w.r.t. mean transit time, T0
    
    Inputs: 
    - t: time series of length N observations; np.array [day]
    - K: RV semi-amplitude [cm/s]
    - P: planet period [days]
    - T0: mean transit time [day]
    
    Output: 
    - array of partial derivatives of length N, where N is number of observations (ie. length of the model vector)
    """
    
    normalized_t = t-T0
    #print(normalized_t)
    arg = (2*np.pi/P) * normalized_t
    #print(arg)
    return (2*np.pi*K/P) * np.cos(arg)

def partial_mu(t, theta, wrt):
    """
    Unpack parameters and calculate all partial derivatives of the model w.r.t. those parameters
    
    Inputs: 
    - t: time series of length N observations; np.array [day]
    - theta: planet orbital parameters; eg. [K, P, T0]; np.array
    - wrt: parameter with respect to which the model's partial derivative is taken
    
    theta breakdown:
    - K: RV semi-amplitude [cm/s]
    - P: planet period [days]
    - T0: mean transit time [day]
    
    Output:
    - derivative vector of length N
    """

    # unpack parameters
    K, P, T0 = theta[0], theta[1], theta[2]
    
    # calculate partial derivative depending on 'wrt'
    normalized_t = t-T0
    arg = (2*np.pi/P) * normalized_t
    
    if wrt=='K':
        return -K * np.cos(arg) * arg/P
    elif wrt=='P':
        return -K * np.cos(arg) * arg/P
    elif wrt=='T0':
        return (2*np.pi*K/P) * np.cos(arg)
    
    print("parameter not recognized")
    
    return                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

def calculate_fim(t, sigma, theta):
    """
    Calculate the generalized Fisher Information Matrix according to I_m,n in 
    https://en.wikipedia.org/wiki/Fisher_information#Multivariate_normal_distribution
    
    Inputs: 
    - t: time series of length N observations; np.array [day]
    - sigma: RV measurement uncertainties associated with each observation; np.array of length N [cm/s]
    - theta: planet orbital parameters; eg. [K, P, T0]; np.array
    
    theta breakdown:
    - K: RV semi-amplitude [cm/s]
    - P: planet period [days]
    - T0: mean transit time [day]
    
    Output:
    - Fisher Information Matrix: len(theta)xlen(theta) matrix; np.array
    
    """

    #print("partial mu, partial K: ", partial_mu(t, theta, 'K'))
    #print("partial mu, partial P: ", partial_mu(t, theta, 'P'))
    #print("partial mu, partial T0: ", partial_mu(t, theta, 'T0'))
    
    fim = np.zeros((len(theta),len(theta)))
    for i in range(len(theta)):
        for j in range(len(theta)):
            #param_i = theta[i]
            #param_j = theta[j]
            wrt_i = param_list[i]
            wrt_j = param_list[j]
            
            factor1 = partial_mu(t, theta, wrt_i).reshape((1,4)) # partial derivative of the model wrt row parameter, transposed
            #print(factor1)
            factor2 = 1/sigma**2 # inverse of covariance matrix (or just the RV variance if held constant)
            factor2[factor2 == np.inf] = 0 # turn infs into zeros (shouldn't it be nans?)
            #print(factor2)
            factor3 = partial_mu(t, theta, wrt_j).reshape((4,1)) # partial derivative of the model wrt column parameter
            #print(factor3)
            #print("shapes: ", factor1.shape, factor2.shape, factor3.shape)
            #print(np.dot(np.dot(factor1, factor2), factor3))
            fim[i][j] = np.dot(np.dot(factor1, factor2), factor3)
    
    return fim

In [28]:
test_t = np.array([2340, 2350, 2380, 2400]) # in Barycentric Julian Dates
test_T0 = 2360 # BJD
test_K = 500 # 5 m/s to cm/s
test_P = 10 # days
test_sigma = np.diag(np.ones(len(test_t))*30) # cm/s
print(test_t.shape, test_sigma.shape)

(4,) (4, 4)


In [29]:
test_theta = [test_K, test_P, test_T0]
calculate_fim(test_t, test_sigma, test_theta)

/tmp/ipykernel_2039244/3248522076.py:132: RuntimeWarning: divide by zero encountered in true_divide
  factor2 = 1/sigma**2 # inverse of covariance matrix (or just the RV variance if held constant)


array([[2741.55677808, 2741.55677808, -328.98681337],
       [2741.55677808, 2741.55677808, -328.98681337],
       [-328.98681337, -328.98681337,  438.64908449]])